# Introduction to functional magnetic resonance imaging (fMRI): loading and plotting data in Python and PyCortex	



## Introduction to functional magnetic resonance imaging (fMRI)

In this session, we will learn about fMRI data properties by loading, manipulating and visualizing it.

# Goals for today

We will go over some important concepts of data manipulation and visualization in fMRI, including: 

* Displaying a data (functional volumes) as an image (`plt.imshow()`)
* Selecting an appropriate colormap for data visualization
* Displaying a 3D array as a mosaic or contact sheet of images
* Plotting timecourses of fMRI data
* Masking / unmasking of fMRI data

By the end, we will have written functions to:

* Display a 3D array as a mosaic of images
* Unmask an array
* Normalize a timeseries

## Short overview of fMRI 

Functional Magnetic Resonance Imaging, or fMRI, is a measure of brain activity over time. FMRI data is acquired using a special pulse sequence designed to measure changes in the magnetic properties of the blood flow using the Magnetic Resonance Imaging (MRI) scanner.

![alt text](../../data/images/mri_scanner.png "mri_scanner")


The functional signal we measure with fMRI is *not* an electrical neural signal (as in EEG, ECoG, or electrophysiology). It is a magnetic signal related to the properties of brain tissue, and it is dominated by blood flow. Blood flow is related to neural activity, because firing neurons need oxygen. The process of firing involves letting electrically charged ions into a cell and actively pumping them back out again, which is metabolically demanding. So once a region of the brain becomes active (once the neurons start firing), metabolism in that region is high, oxygen gets stripped off of hemoglobin molecules in red blood cells in the area (thereby changing the magnetic properties of hemoglobin, creating a deoxyhemoglobin). This initiates a complex process to increase blood flow to the electrically active area. 

![alt text](../../data/images/deoxyhemoglobin.png "deoxyhemoglobin.png")

The specific mechanisms that lead from neural activity to changes in blood flow are (a) not well understood, and (b) beyond the scope of this class. For now, just know that there are several ways to measure functional responses with MRI, and the specific one that we work with is the Blood Oxygenation Level Dependent Response, or the BOLD response. 

### fMRI has high spatial but low temporal resolution

We have belabored this here a little because this complexity should always be a source of humility for anyone working with fMRI or trying to interpret fMRI results. It is an **indirect, slow measure** - and these considerations strongly constrain the kinds of experiments you can do with fMRI and the conclusions you can draw from those experiments.

An important practical upshot of this for our purposes is that the signal we measure changes much more slowly than the signal we measure in EEG or ECoG. Responses emerge over seconds, not milliseconds.

![alt text](../../data/images/imaging_modalities.png "imaging_modalities.png")

                                      Sejnowski et al., Nature Neurosci., 2014




One fMRI image (fMRI volume) is acquired for a given unit of time called a repetition time (TR). A TR is typically 1-2 seconds *(usually 1.0, 1.5, or 2.0 seconds)*. Every image records the activity in the brain at a given point in time. The following image shows a single volume of fMRI data (one two-second snapshot of brain activity).

![alt text](../../data/images/fig1slice.png "fig1")

The dimensions of the brain volume measured by fMRI can vary. Each individual fMRI measurement unit is called a *voxel*, which is short for volumetric pixel. The voxels in this data are about 2.4 x 2.4 x 4.0 mm (X x Y x Z) in size. 

### FMRI activity in time

Once a neural event is triggered by a stimulus presentation the vascular system needs to respond to the need for glucose and oxygen in that specific brain area. This can take up to 1-2 seconds. Hence the hemodynamic response lags the triggered event by 1-2 seconds, which peaks around 5 seconds after the stimulus onset.

![alt text](../../data/images/lagged_activity.png "lagged_activity.png")

### Example fMRI experiments 

#### Viewing natural images 

![alt text](../../data/images/fmri_example_experiment.png "fmri_example_experiment.png")

#### Fusiform Face Area (FFA)

![alt text](../../data/images/ffa.png "ffa.png")


## FMRI as a *mapping* technique
The scanning volume for the data we will use here consists of 30 transverse slices (i.e. Z is between 0 and 29). Transverse slices are horizontal, i.e., approximately parallel to the plane of the eyes and ears. Each slice corresponds to a 2D image of 100 x 100 voxels.

We can measure fMRI responses across the whole brain. Therefore fMRI can be seen as a *mapping* technique. In order to make brain maps, we have to be able to match measured voxel responses to the participant's brain anatomy. Using a different type of MRI sequence, we can collect a **structural scan** (also called **anatomical scan**), and obtain a high resolution image of a participant's brain.


![alt text](../../data/images/MPRAGE.png "MPRAGE.png")

![alt text](../../data/images/MPRAGE_wcortex.png "MPRAGE_wcortex.png")

Further, fMRI studies often focus on the cerebral cortex (the outermost layer of the brain). Consequently, it is common to display the results of statistical analyses of fMRI data on inflated and flattened representations of the cerebral cortex. Such cortical surface maps provide a way to examine all cortical fMRI data at once, with the anatomical location of the functional data made clear. 

The cortical surface must be computationally extracted from high spatial resolution anatomical MRI scans, and often manually edited (*NOTE: Manual editing to create a good corical surface can take days or weeks of effort! This data is not free!*)

![alt text](../../data/images/cortex_3views.png "cortex_3views.png")



## Storing fMRI data for data analysis

We store fMRI data as a matrix. This means that each volume (a timepoint) in the experiment will correspond to a row in the matrix, and each voxel will correspond to a column in that matrix. For this reason, we need to make sure the criteria we use to move each 3D image to a matrix row is preserved and this operation is inverted. Let's look at an example.

![alt text](../../data/images/fmri_dimensions.png "fmri_dimensions.png")


## Loading data

fMRI data is stored in a variety of formats. The medical imaging community has a standardized image format called **Digital Imaging and Communications in Medicine (DICOM)** to handle and store raw medical imaging data. When data is collected using the MRI scanner each volume is stored in a DICOM file that contains both a header and the image data. The DICOM header stores useful information about the participant's name, pulse sequence, the type of scan, image dimensions, etc. (Another popular medical imaging format is the Analyze format, where the image and header file are stored separately.) You can read about these file formats [here](http://people.cas.sc.edu/rorden/dicom/index.html).

Before we start analyzing the data we convert the raw DICOM files into the commonly used **Neuroimaging Informatics Technology Initiative (nifti)** file format. Files stored in this format usually have the extension .nii or .nii.gz. 

We will use the `nibabel` python module to load data that is stored such data formats.

In [ ]:
import nibabel
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt  # for visualization

%matplotlib widget
%config InlineBackend.figure_format = 'retina'

In the following we will load one run (also referred to as a scan) worth of fMRI data that was stored as a nifti file format. 

In [ ]:
# Create a nifti (nii) proxy object
fname = '../../data/fMRI/categories/s01_categories_01.nii.gz'
nii = nibabel.load(fname) 

# This object stores the infomation *about* the fMRI data stored in the file. 
# This meta-data can be accessed via attributes of the `nii` object.
print('nii.in_memory : ', nii.in_memory)
print('nii.shape : ', nii.shape)

In [ ]:
# Retrieve actual data as an array
data = nii.get_fdata()
print('nii.in_memory : ', nii.in_memory)
print('data shape : ', data.shape)

The dimensions of the data are (X, Y, Z, T) (T is time, in TRs). Thus, there are 120 volumes (120 time points). Each volume has 30 horizontal or transverse slices with 100 x 100 pixels.

![alt text](../../data/images/slices.png "slices.png")


#### Transpose data

When we work with functional images it is in general more convenient (for reasons like averaging over time, transfering data to a standard units, etc.) to have the data in T, Z, Y, X format. This is the opposite of the conventions you've seen so far for EEG / ECoG data. 

When we worked with EEG/ECoG data in the first half of the class, time was the *last* axis; here, time is the *first* axis. The reasons for this convention will become more obvious as we go, and we see how this convention makes for convenient syntax and shortcuts. 

Hence, we will use the `transpose` function  of the numpy package (or `.T` method of numpy arrays) to make this dimensional switch.

In [ ]:
# Transpose the data
print(data.shape)
dataT = np.transpose(data)

# Or, equivalently
dataT_ = data.T

print("dataT shape :", dataT.shape)
print("dataT_ shape : ", dataT_.shape)

#### assert function
You can check whether to variables are the same using the `assert` function.

In [ ]:
a = 2 
b = 1
assert a==b

You can also append an error message. This message will be printed if the two are not equal

In [ ]:
assert a==b, 'these two are not equal!'

### Breakout session: Explore the function transpose

> Now test whether dataT and dataT_ are the same using assert

In [ ]:
# Are dataT and dataT_ the same?
### STUDENT ANSWER

## Exploring the data

For fMRI, the functions we will use to explore the data are in general more basic than the functions we used in the first part of the class. We will not use a single module like MNE to make standard plots; we will construct our own!

This is a little more work, but is very generalizable to other kinds of analyses, and if you get good at it you can make exactly the plot of your data that you want.

One of the first questions about a data set (after its size) that you should explore is 

    "What is the scale (min/max) of the data?". 

Afterwards you may be interested to know its mean, standard deviation, and how in general the data looks like and make a histogram plot of the data.

In [ ]:
# Keep data transposed
data = dataT

del dataT, dataT_
print(data.shape)

In [ ]:
# Find out the scale of the data
print(np.min(data), ', ', np.max(data)) 

# Find out its mean
print(np.mean(data))

# Find out its mean
print(np.std(data))

In [ ]:
# Make a histogram of the data
print(data.shape)
print(data.flatten()[:10])
_ = plt.hist(data.flatten(), bins=50)


> - What does this tell you about the data? 

> - What are the axes on this plot? 

### Plot the timecourse of a single voxel
Now we can plot the timecourse for one voxel somewhere in the middle of the brain (e.g. at Z=10, Y=34, X=34)

In [ ]:
plt.figure()
_ = plt.plot(data[:, 10, 34, 34])
plt.show()

### Breakout session:
> - Save all transverse slices for a sagittal and coronal axis (you can choose any number), in a variable called `transverse``

> - Make a plot for all transverse slices (different voxels are plotted in the same figure). Label the axis. What does this plot tell us?

> - Transpose the transverse variable and plot. Label the axis accordingly. What does this plot tell us?

In [ ]:
### STUDENT ANSWER


Note that we have 30,000 measurements to plot like this. So we *could* make a plot like the eeg representations that we had, but those were pretty busy even with 60 lines. So, instead, we will view our data as images.

## Displaying data as an image

First, we will get a broader view of the first volume of our data. The (T, Z, Y, X) dimension ordering that we have for the data makes it easy to select volumes (time snapshots of brain activity).

Below are some ways to select volumes:

In [ ]:
# We can select one volume like this: 
first_volume = data[0, :, :, :]

# Or like this: 
alt_first_volume1 = data[0, ...]

# Or like this: 
alt_first_volume2 = data[0]

# These are all the same1
assert np.all(first_volume==alt_first_volume1)
assert np.all(first_volume==alt_first_volume2)

In [ ]:
print(first_volume.shape)

### Visualizing the horizontal slice
Let's look at an example of a horizontal slice from the first volume. This can be done by selecting one of the slices as follows:

In [ ]:
# Z=15 is halfway through the volume we have scanned
slice_horizontal = first_volume[15,:,:]

# You can set the image origin [0,0] to be in the lower left corner
# by using origin='lower'
plt.figure()
im = plt.imshow(slice_horizontal, origin='lower') 
_ = plt.colorbar(im)
plt.show()

# Alternative:
plt.figure()
im = plt.imshow(slice_horizontal[::-1]) 
_ = plt.colorbar(im)
plt.show()

### Breakout session:
> - Plot other slices to see how the shape of the brain is different
> - Change the properties of the figure. Explore the keyword arguments for imshow, see what each does! (hints: show axes, change colormap, what about vmin and vmax values, set those)

colormaps: https://matplotlib.org/stable/users/explain/colors/colormaps.html

In [ ]:
### STUDENT ANSWER
# You can set the image origin [0,0] to be in the lower left corner
# by using origin='lower'
plt.figure()
im = plt.imshow(slice_horizontal, origin='lower', cmap='inferno', vmax=1500) 
_ = plt.colorbar(im)
plt.show()

### Breakout session:
> Write a small helper function that takes a slice number as an input returns the data (2D array) of that slice
> Plot this image

In [ ]:
### STUDENT ANSWER
def select_slice(slice_number):
    pass

In [ ]:
slice_number = 12

plt.figure()
im = plt.imshow(select_slice(slice_number), origin='lower', cmap='inferno') 
_ = plt.colorbar(im)
plt.show()

### Breakout session: Visualizing sagittal and coronal views
We can also slice the brain on different axis to obtain sagittal and coronal slices. 

> Try to plot these below. We have not told you which dimension corresponds to slicing the brain in a sagittal or coronal view. You should try the different axes and find it out.

In [ ]:
# Select a sagittal slice and plot
### STUDENT ANSWER
def select_slice_coronal(slice_number):
    pass
    
slice_number = 45

plt.figure()
im = plt.imshow(select_slice_coronal(slice_number), origin='lower', cmap='inferno') 
_ = plt.colorbar(im)
plt.show()

In [ ]:
# Select a coronal slice and plot
### STUDENT ANSWER

def select_slice_sagittal(slice_number):
     pass
    
slice_number = 70

plt.figure()
im = plt.imshow(select_slice_sagittal(slice_number), origin='lower', cmap='inferno') 
_ = plt.colorbar(im)
plt.show()

### Changing matplotlib default parameters

You can set the default colormap, default interpolation or many other parameters in `matplotlib.rcParams`.

For example to set all the colormaps in this `ipython` session to the colormap 'viridis' we can use the following line:
    * `matplotlib.rcParams['image.cmap'] = 'viridis'` # or whatever your favorite map is

In [ ]:
import matplotlib
matplotlib.rcParams['image.cmap'] = 'viridis' # or whatever your favorite map is e.g. 'gray', 'hot'
matplotlib.rcParams['image.interpolation'] = 'nearest'

An alternative way to change a figure's properties is to create a dictionary of keywords that can be used as a  keyword argument to the `imshow` function.

This has the advantage of not setting the default parameters. Yet, we can easily change a number of parameters in the `imshow` function by just passing the keywords dictionary to the `imhsow` function. The following cell is demonstrating this:

In [ ]:
plt.figure()
im_kws = dict(aspect='auto', vmin=0, vmax=2000, cmap='hot', interpolation='nearest') 
plt.imshow(first_volume[:,  30, :], **im_kws)
plt.colorbar()
plt.show()

### Plot all horizontal slices

Let's try to make a plot with all of the horizontal slices, so we can see one entire 3D volume at once. For this, we will use the `subplot()` function in matplotlib:

In [ ]:
fig = plt.figure(figsize = (10,10))
n_slices = 30
nrows, ncols = 5, 6
for s in range(n_slices):
    ax = fig.add_subplot(nrows, ncols, s+1)
    slice_horizontal = first_volume[s,:,:]
    plt.imshow(slice_horizontal)
    ax.axis('off')
    

Now let's make a function that plots the above:

In [ ]:
def plot_horizontal_slices(vol, **kwargs):
    fig = plt.figure(figsize=(10,10))
    nslices = vol.shape[0]
    subplot_size = int(np.ceil(np.sqrt(nslices)))
    for s in range(nslices):
        ax = fig.add_subplot(subplot_size, subplot_size, s+1) 
        slice_horizontal = vol[s,:,:]
        plt.imshow(slice_horizontal, **kwargs)
        ax.axis('off')

### Breakout session:
> Call the above function (plot_horizontal_slices), and try out different plotting arguments (e.g. change the colormap, change the interpolation, etc.):

In [ ]:
### STUDENT CELL

## Masking

You may have noticed in the figure above that many of the voxels do not overlap with the brain (or more specifically the gray matter in the cortex) at all. Actually, let's try to plot some of those voxels:

In [ ]:
plt.figure()
plt.plot(data[:,0,0,0], label='Edges of volume, 1')
plt.plot(data[:,1,1,1], label='Edges of volume, 2')
plt.plot(data[:,-1,-1,-1], label='Edges of volume, 3')

# A middle brain voxel
plt.plot(data[:,10,34,34], label='Mid brain')
plt.legend(loc='best')
plt.show()

The edge-of-scan voxels are clearly not in the brain, however, they show some variance due to noise. It is the practice in fMRI to mask out, or zero out these voxels.

A mask is a 3D binary array that is derived from the high resolution anatomical scan of the subject. The mask indicates, for every voxel in the 30 x 100 x 100 matrix, which ones should be ignored and which should be kept. Let's load and look at a voxel mask for this subject:

In [ ]:
# Load a stored mask for this subject
fname = '../../data/fMRI/categories/s01_categories_mask_cortical.npz'
mask = np.load(fname)['mask']
print('Mask shape: ', mask.shape)
print(mask[10,:,:])

Let's plot one fo the slices below:

In [ ]:
plt.figure()
mask_horizontal = mask[15,:,:]
plt.imshow(mask_horizontal, origin='upper', cmap='gray')
_ = plt.axis("off")
plt.show()

We can see that the mask clearly indicates which voxels should be kept. 

### Breakout session: 
> Let's look at the 3D structure. Use the function we defined above to plot the entire 3D mask:

In [ ]:
### STUDENT ANSWER

You can see how now we have a mask that indicates which voxels we should keep and which we should exclude.

We can use the mask to set the tiny values outside the brain, and the values in the middle of the brain (in subcortical stuctures) to zero:

In [ ]:
# Let's plot the horizonral slices for the first volume
plot_horizontal_slices(first_volume)

In [ ]:
masked_vol_0 = first_volume * mask

# Now plot masked_vol_0:
plot_horizontal_slices(masked_vol_0)

You can see that the voxels outside the cortex (including in the middle of the brain) are zeroed out now! Let's apply this mask to the entire dataset now and zero out the same voxels in each volume:

In [ ]:
# First create an array with zeros
print(data.shape)
masked_data = np.zeros_like(data)

# NOTE: np.zeros_like is similar to creating an array using np.zeros. We can check this by:
assert np.all(np.zeros(data.shape)==np.zeros_like(data)) 

# Not run through the entire data set and mask each volume
for i in range(data.shape[0]):
    masked_data[i,:,:,:] = data[i,:,:,:] * mask

We can replace the above foor loop using list comprehension.


In [ ]:
first_volume = data[0]
print('shape of first_volume is {}'.format(first_volume.shape))

first_volume_masked = first_volume[mask==True]  # we can omit ==True, because mask is already a boolean array
print('shape of first_volume_masked is {}'.format(first_volume_masked.shape))

In [ ]:
plt.figure()
_ = plt.hist(first_volume_masked.flatten(), 50)
plt.show()

Now we can compress the entire matrix:

In [ ]:
masked_data_v3 = data[:, mask]

print('masked_data_v3 is a {0} of size {1}'.format(type(masked_data_v3), masked_data_v3.shape))
print('each element is a {0} of size {1}'.format(type(masked_data_v3[0]), masked_data_v3[0].shape))

If we make another histogram of the masked (collapsed) data, we can see that we have a more interesting range of values left:

In [ ]:
plt.figure()
_ = plt.hist(masked_data_v3.flatten(), 50)
plt.show()

We can easily go back and forth between the two formats, we need to have the mask and the 2D matrix to produce the 4D matrix:

In [ ]:
unmasked_data = np.zeros(data.shape)
unmasked_data[:, mask] = masked_data_v3 

print('unmasked_data is a {0} of size {1}'.format(type(unmasked_data), unmasked_data.shape))
print('each element is a {0} of size {1}'.format(type(unmasked_data[0]), unmasked_data[0].shape))

In [ ]:
plt.figure()
_ = plt.hist(unmasked_data.flatten(), 50)
plt.show()


## Time series

Remember that one of these volumes is acquired at every time unit. The time unit here is 2.0045 seconds. Let's look at one slice at different time points. Now because masked_data_v3 only has cortical voxels, we can plot any of its dimension knowing that we are looking at gray matter.

In [ ]:
masked_data_v3.shape

Let's plot the activity in time for different voxels. What do you notice?

In [ ]:
TR = 2.0045
n_points = 100
time_points = np.arange(n_points)*TR

plt.figure(figsize=(10,5))
plt.plot(time_points, masked_data_v3[:n_points, 4].T)
_ = plt.xlabel("Time (s)")
_ = plt.ylabel("fMRI activity")

Let's plot four different voxels time series

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(time_points, masked_data_v3[:n_points, 4].T)
plt.plot(time_points, masked_data_v3[:n_points, 10].T)
plt.plot(time_points, masked_data_v3[:n_points, 100].T)
plt.plot(time_points, masked_data_v3[:n_points, 1000].T)

_ = plt.xlabel("Time (s)")
_ = plt.ylabel("fMRI activity")

The voxels seem to have a different baseline! (You might have suspected this based on the image plots or the histogram plots above, too).

Let's plot one slice at different time points:

In [ ]:
fig = plt.figure(figsize = (10,10))
z_slice = 10
for s in range(15):
    ax = fig.add_subplot(5, 5, s+1)
    slice_horizontal = data[s, z_slice, :, :]
    slice_horizontal[~mask[z_slice, : , :]] = 0
    plt.imshow(slice_horizontal)
    plt.title('TR #{n}'.format(n=s+1))
    ax.axis('off')

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
ax.imshow(masked_data_v3, aspect='auto')
# Set the axis labels using the `setp` method.
# http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.setp
plt.setp(ax, xlabel='Voxels', ylabel='Time')

### Normalize the activity at each voxel (zscore across time)

You can see the same effect in the time plots as well as in the mosaic plots: some voxels have a different baseline than others. 

We need to normalize the activity of each voxel in time to be able to see local fluctuations in the signal. This normalization is also called *z-score* or *standard score*.

1. We will first take the mean and standard deviation across time for each cortical voxel.
2. For each voxel, we will substract the mean from each time point.
3. For eacl voxel, we will divide each time point by the standard deviation.

In [ ]:
print(masked_data_v3.shape)
data_norm = masked_data_v3 - masked_data_v3.mean(axis = 0)
data_norm = data_norm / data_norm.std(axis = 0)

Now plot the time course:

In [ ]:
plt.figure(figsize = (10,5))
TR = 2.0045
points = range(0,100)
time_points = np.array(points)*TR

plt.plot(time_points, data_norm[:n_points, 4].T)
plt.plot(time_points, data_norm[:n_points, 10].T)
plt.plot(time_points, data_norm[:n_points, 100].T)
plt.plot(time_points, data_norm[:n_points, 1000].T)

_ = plt.xlabel("Time (s)")
_ = plt.ylabel("fMRI activity")

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
ax.imshow(data_norm, aspect='auto') 
plt.setp(ax, xlabel='Voxels', ylabel='Time')

And plot the volumes in time:

In [ ]:
data_norm_unmasked = np.zeros_like(data)
data_norm_unmasked[:,mask] = data_norm

In [ ]:
fig = plt.figure(figsize = (10,10))
for s in range(15):
    ax = fig.add_subplot(5,5,s+1)
    slice_horizontal = data_norm_unmasked[s,10,:,:]
    slice_horizontal[~mask[10,:,:]] = 0
    plt.imshow(slice_horizontal, vmin=-3, vmax=3) # don't forget vmin / vmax!
    ax.axis('off')

## Visualization using pyCortex
 
We will now work with a visualization tool called [pycortex](https://gallantlab.org/pycortex/).


## Alignment
fMRI measures activity across the whole brain (or large swathes of the brain), but it is difficult to map that activity to specific anatomical locations in the brain based on functional data alone. Many factors make anatomy ambiguous in functional data: **(1)** functional data are low resolution; **(2)** functional slices are not always aligned with the head in a consistent or precise way across subjects; and **(3)** there is considerable variation in anatomy (i.e., specific locations of sulci & gyri) across subjects. Thus, a common step in fMRI processing is to align the functional data with a high-resolution anatomical scan of the same subject. 

### Alignment in principle
The scanner stores the location of the slices with respect to the magnet *isocenter* (the strongest point of the magnetic field in the MRI scanner; the center of the magnet). This data is transferred to the nifti files at file creation time. These transformations (from data space to scanner space) are still not enough to precisely align data from different anatomical and functional scans. The functional and anatomical data may have been collected on different days, or the subject may have shifted slightly between scans. Thus, the functional data needs to be moved and/or rotated in 3D to make sure it aligns as precisely as possible with the underlying anatomical data. 


![alt text](../../data/images/fMRI_Transforms.001.png "fMRI_Transforms.001.png")


### Alignment in pycortex
pycortex allows manual adjustment of the rotation & position of the functional data with respect to the 3D surface.

![alt text](../../data/images/pycortex_aligner_transverse.png "pycortex_aligner_transverse.png")



In [ ]:
# Load some necessary libraries
import os
import nibabel  # Neuroimaging library to work with MR images
import cortex  # This is our mapping software pycortex
# import neurods # Our class module
import numpy as np
from scipy.stats import zscore
import matplotlib.pyplot as plt

# Set matplotlib defaults
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.aspect'] = 'auto'
plt.rcParams['image.cmap'] = 'viridis'

%matplotlib widget
%config InlineBackend.figure_format = 'retina'

# Load data

Same as previously: load using nibabel, use get_data() method of the nibabel data object, transpose resulting data array, and zscore the data. For now, we will treat this as a generic data set (next week, we will learn more about the experiment that generated this data as we begin to actually analyze the data).

In [ ]:
fname = '../../data/fMRI/categories/s01_categories_01.nii.gz'
nii = nibabel.load(fname) 
voldata = nii.get_fdata().T

mask_fname = '../../data/fMRI/categories/s01_categories_mask_cortical.npz'
mask = np.load(mask_fname)['mask']
data = voldata[:, mask]
data = zscore(data, axis=0)

In [ ]:
first_volume = voldata[0]
print("Dimensions of the first volume: {0}".format(first_volume.shape))
print("Dimensions of the data: {0}".format(data.shape))

That is: (time, Z, Y, X). 

# Onward to 3D data visualizations! 

We will use a python module called pycortex to show data in 3D on the brain. This module was developed here at UC Berkeley in the Gallant lab, mostly by James Gao, with help from Alex Huth, Mark Lescroart, and other lab members. The code is freely available online [here](https://github.com/gallantlab/pycortex), and a paper summarizing the code can be found [here](http://journal.frontiersin.org/article/10.3389/fninf.2015.00023/full). 

To map the functional data onto the cortex, pycortex requires at least two things:

1. The cortical surface of the subject. 
    * pycortex stores cortical surface files (and several other files) for each subject in a reliably structured directory of files. Because of this reliable directory structure, all we need to provide to the code is a subject ID string, and the code will be able to find and load the relevant cortical surface files. 
2. The functional to anatomical aligmnent of this data to that cortical surface
    * Alignment of functional data to anatomical data proceeds by an *affine transform*. How this transformation works is beyond the scope of this class, but you can look it up on [wikipedia](https://en.wikipedia.org/wiki/Affine_transformation) or in your favorite linear algebra textbook if you're curious. Or here: https://andysbrainbook.readthedocs.io/en/latest/fMRI_Short_Course/Preprocessing/Registration_Normalization.html
    * The practical upshot is that a 4x4 matrix of numbers is sufficient to store the 3 rotations (around the x, y, and z axes) and 3 the transformations (in the x, y and z dimensions) that will transform the functional data in space such that they are aligned with the anatomical data (with the cortical surface). In the pycortex code, "transform" is abbreviated in variable names as `xfm`. Just as with the cortical surface, we only need to specify a name for a transform, and the code will know where to find the file that contains the affine transformation matrix. 

In [ ]:
# (1) subject (specifies the cortical surface of the brain)
subject = 's01' 
# (2) transform 
transform = 'catloc' 

To display data in pycortex, we need to create a pycortex object that contains all the relevant information: the data, the cortical surface (i.e., the subject) and the affine transform. That object is called a Volume.

In [ ]:
# Create a volume
data_volume = cortex.Volume(first_volume, subject, transform) 
print(data_volume)

In [ ]:
# Show the volume on a flatmap (ooooh)
# cortex.webgl.show(data_volume)
_ = cortex.quickflat.make_figure(data_volume)


In [ ]:
# Show a flatmap inline
# (If you get warnings about a module called shapely, ignore them; they are not important.)
_ = cortex.quickflat.make_figure(data_volume)

## Masks
pycortex can be used to generate cortical masks; this is where the mask we used above came from.

In [ ]:
mask = cortex.db.get_mask(subject, transform, type='cortical')
first_volume_masked = voldata[0, mask]
print('Shape of mask: ', mask.shape)
print('Shape of masked data: ', first_volume_masked.shape)

pycortex can also directly visualize masked data

In [ ]:
# Show creation of volume from masked data
print(data.shape)
kwargs = dict(vmin=-3, vmax=3)
vol = cortex.Volume(data[0], subject, transform, mask=mask, **kwargs)
print(vol)
print(vol.shape)

In [ ]:
# Display this volume, too
_ = cortex.quickflat.make_figure(vol)
# Select other timepoints to display


In [ ]:
# Create a movie volume
data_movie = cortex.Volume(data, subject, transform, mask=mask, vmin=-3, vmax=3)

In [ ]:
### TEACHER INFO
# show 3D web display with a 4D volume
# cortex.webgl.show(data_movie)

---